# Ejemplo de análisis exploratorio de datos (pt. 2)

## Diagnóstico de calidad de los datos

Esta fase tiene como propósito ~modificar~ obtener el conjunto de datos (original) tratando aquellos datos que puedan derivar conclusiones erróneas o sesgadas.

Se compone de tareas tales como,
- Identificar y __tratar los registros duplicados__
- __Disminuir la cardinal__ de aquellas variables que no aportan diferenciación significativa
- Detectar y __tratar _outliers___
- Identificar y __tratar registros erróneos, inexistente o incompletos__

### Tratamiento de registros duplicados

Esta tarea tiene como propósito:
- Asegurar que los datos que no están duplicados
- Asegurar que las columnas no tengan un alto número de duplicados, ya que podrían no aportar información

In [1]:
import pandas as pd

vehiculos = pd.read_csv("./out/vehiculos-1-procesado_inicial.csv")
vehiculos

,fabricante,modelo,year,desplazamiento,cilindros,transmision,traccion,clase,combustible,rendimiento,co2
0,AM General,DJ Po Vehicle 2WD,1984,2.5,4.0,Automatic 3-spd,2-Wheel Drive,Special Purpose Vehicle 2WD,Regular,17,522.764706
1,AM General,DJ Po Vehicle 2WD,1984,2.5,4.0,Automatic 3-spd,2-Wheel Drive,Special Purpose Vehicle 2WD,Regular,17,522.764706
2,AM General,FJ8c Post Office,1984,4.2,6.0,Automatic 3-spd,2-Wheel Drive,Special Purpose Vehicle 2WD,Regular,13,683.615385
3,AM General,FJ8c Post Office,1984,4.2,6.0,Automatic 3-spd,2-Wheel Drive,Special Purpose Vehicle 2WD,Regular,13,683.615385
4,AM General,Post Office DJ5 2WD,1985,2.5,4.0,Automatic 3-spd,Rear-Wheel Drive,Special Purpose Vehicle 2WD,Regular,16,555.437500
...,...,...,...,...,...,...,...,...,...,...,...
38431,smart,fortwo electric drive coupe,2011,NaN,NaN,Automatic (A1),Rear-Wheel Drive,Two Seaters,Electricity,87,0.000000
38432,smart,fortwo electric drive coupe,2013,NaN,NaN,Automatic (A1),Rear-Wheel Drive,Two Seaters,Electricity,107,0.000000
38433,smart,fortwo electric drive coupe,2014,NaN,NaN,Automatic (A1),Rear-Wheel Drive,Two Seaters,Electricity,107,0.000000
38434,smart,fortwo electric drive coupe,2015,NaN,NaN,Automatic (A1),Rear-Wheel Drive,Two Seaters,Electricity,107,0.000000


Se debe establecer una entidad (_variable auxiliar_) que permita detectar registros duplicados. Recordar que esta entidad quedó definida en la fase 1 (Identificación de los datos):
- __fabricante + modelo + año__


[`Series.str.cat`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.cat.html), [`Series.apply`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.apply.html)

In [2]:
vehiculos['modelo_unico'] = vehiculos['fabricante'].str.cat([vehiculos['modelo'], vehiculos['year'].apply(str)],sep='-')
vehiculos['modelo_unico'].value_counts() # contador de entidades

modelo_unico
Jeep-Cherokee/Wagoneer-1985               24
Ford-F150 Pickup 2WD-1984                 19
GMC-C15 Pickup 2WD-1984                   19
Chevrolet-C10 Pickup 2WD-1984             19
GMC-C15 Pickup 2WD-1985                   18
                                          ..
Mercedes-Benz-C300-2015                    1
Mercedes-Benz-C300-2009                    1
Mercedes-Benz-C280 4matic-2007             1
Mercedes-Benz-C280 4matic-2006             1
smart-fortwo electric drive coupe-2016     1
Name: count, Length: 17448, dtype: int64

Una lectura rápida de los valores obtenidos (cuenta) permite detectar registros duplicados o, por otra parte, identificar la existencia de un número mayor de atributos que complementarían la definición de la entidad. 

Por ejemplo, a continuación se muestra el detalle para la entidad definida por: __marca-modelo-año__:

- `Chevrolet-C1500 Pickup 2WD-1991`

In [3]:
vehiculos[ vehiculos['modelo_unico'] == 'Chevrolet-C1500 Pickup 2WD-1991' ]

,fabricante,modelo,year,desplazamiento,cilindros,transmision,traccion,clase,combustible,rendimiento,co2,modelo_unico
4957,Chevrolet,C1500 Pickup 2WD,1991,4.3,6.0,Automatic 4-spd,Rear-Wheel Drive,Standard Pickup Trucks,Regular,17,522.764706,Chevrolet-C1500 Pickup 2WD-1991
4958,Chevrolet,C1500 Pickup 2WD,1991,4.3,6.0,Manual 4-spd,Rear-Wheel Drive,Standard Pickup Trucks,Regular,17,522.764706,Chevrolet-C1500 Pickup 2WD-1991
4959,Chevrolet,C1500 Pickup 2WD,1991,4.3,6.0,Manual 5-spd,Rear-Wheel Drive,Standard Pickup Trucks,Regular,17,522.764706,Chevrolet-C1500 Pickup 2WD-1991
4960,Chevrolet,C1500 Pickup 2WD,1991,4.3,6.0,Manual 5-spd,Rear-Wheel Drive,Standard Pickup Trucks,Regular,17,522.764706,Chevrolet-C1500 Pickup 2WD-1991
4961,Chevrolet,C1500 Pickup 2WD,1991,5.0,8.0,Automatic 4-spd,Rear-Wheel Drive,Standard Pickup Trucks,Regular,15,592.466667,Chevrolet-C1500 Pickup 2WD-1991
4962,Chevrolet,C1500 Pickup 2WD,1991,5.0,8.0,Manual 5-spd,Rear-Wheel Drive,Standard Pickup Trucks,Regular,16,555.437500,Chevrolet-C1500 Pickup 2WD-1991
4963,Chevrolet,C1500 Pickup 2WD,1991,5.7,8.0,Automatic 4-spd,Rear-Wheel Drive,Standard Pickup Trucks,Regular,15,592.466667,Chevrolet-C1500 Pickup 2WD-1991
4964,Chevrolet,C1500 Pickup 2WD,1991,5.7,8.0,Manual 4-spd,Rear-Wheel Drive,Standard Pickup Trucks,Regular,13,683.615385,Chevrolet-C1500 Pickup 2WD-1991
4965,Chevrolet,C1500 Pickup 2WD,1991,5.7,8.0,Manual 5-spd,Rear-Wheel Drive,Standard Pickup Trucks,Regular,14,634.785714,Chevrolet-C1500 Pickup 2WD-1991
4966,Chevrolet,C1500 Pickup 2WD,1991,5.7,8.0,Manual 5-spd,Rear-Wheel Drive,Standard Pickup Trucks,Regular,14,634.785714,Chevrolet-C1500 Pickup 2WD-1991


En el detalle, se puede ver que cada modelo-único tiene diferentes configuraciones de cada vehículo, por ejemplo, transmisión de 4 o 5 velocidades. Por tanto, no corresponden a valores duplicados. 

Para este caso de este ejemplo, se definen como duplicados aquellos registros que son idénticos en todas sus columnas.

[`DataFrame.duplicated()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html) verifica la cantidad de registros duplicados.

In [4]:
vehiculos[vehiculos.duplicated()].shape

(1506, 12)

La consulta identifica 1506 registros duplicados, por lo tanto, son eliminados del análisis debido a que podrían generar sesgos en las conclusiones.

[`DataFrame.drop_duplicates()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html) retorna los registros unicos, eliminando los registros duplicados.

In [5]:
vehiculos.drop_duplicates(inplace=True)
vehiculos.shape

(36930, 12)

A continuación, se elimina la columna auxiliar `modelo_unico` (usada para detectar duplicados).

In [6]:
del vehiculos['modelo_unico']

La siguiente tarea consiste en identificar las variables cuyos registros contengan el mismo valor (cardinalidad).

In [7]:
def duplicados_por_columna(df):
    regs, cols = df.shape
    for atributo in df:
        frecuencia_columna = df[atributo].value_counts()
        max_col = frecuencia_columna.iloc[0]
        min_col = frecuencia_columna.iloc[-1]
        print(f'{round(max_col/regs, 2): <4} > {atributo: <14} > {round(min_col/regs, 3)}')

duplicados_por_columna(vehiculos)

0.1  > fabricante     > 0.0
0.01 > modelo         > 0.0
0.04 > year           > 0.007
0.1  > desplazamiento > 0.0
0.38 > cilindros      > 0.0
0.29 > transmision    > 0.0
0.35 > traccion       > 0.005
0.14 > clase          > 0.0
0.65 > combustible    > 0.0
0.1  > rendimiento    > 0.0
0.08 > co2            > 0.0


Entre los datos presentados anteriormente, los atributos `traccion`, `transmision`, `cilindros` y `combustible` podrían presentar valores repetidos.

Mayor detalle de cada atributo puede ser analizada contando valores (absolutos).

Un análisis del atributo `traccion`:

In [8]:
vehiculos['traccion'].value_counts()

traccion
Front-Wheel Drive             13018
Rear-Wheel Drive              12750
4-Wheel or All-Wheel Drive     6408
All-Wheel Drive                2252
4-Wheel Drive                  1116
2-Wheel Drive                   412
Part-time 4-Wheel Drive         177
Name: count, dtype: int64

También es posible obtener un resultado normalizado (relación respecto del total, %).

In [9]:
vehiculos['traccion'].value_counts(normalize=True)

traccion
Front-Wheel Drive             0.360280
Rear-Wheel Drive              0.352863
4-Wheel or All-Wheel Drive    0.177345
All-Wheel Drive               0.062325
4-Wheel Drive                 0.030886
2-Wheel Drive                 0.011402
Part-time 4-Wheel Drive       0.004899
Name: proportion, dtype: float64

En este caso, el [conocimiento de dominio](https://www.edmunds.com/car-technology/what-wheel-drive.html) podría ayudar a concluir que, probablemente, la tracción `2-Wheel Drive` es simplemente una forma distinta para denominar `Front-Wheel Drive` o `Rear-Wheel Drive`.

__NOTA__: Es útil utilizar gráficos para la identificación de cardinalidad.

En relación con el atributo `transmision`,

In [10]:
import hvplot.pandas

fig = vehiculos['transmision'].value_counts(
    normalize=True).hvplot.barh(
    ylabel='Proporción',
    xlabel='Tipos',
    grid=True,)
fig.opts(
    toolbar='above',
    title='Transmisión',
    active_tools=[],
    height=500, width=600
)

:Bars   [transmision]   (proportion)

Aunque `transmision` tenga una cardinalidad relativamente alta, en realidad muestra una distribución de tipos, con dos clases mayoritarias.

En relación con el atributo `cilindros`:

In [11]:
import hvplot.pandas

fig = vehiculos['cilindros'].hvplot.hist(
    grid = True,
    ylabel = 'Frecuencia',
    xlabel = 'Cilindros'
)
fig.opts(
    title = 'Cilindrada',
    toolbar = 'above',
    active_tools = [],
    width=600
)

:Histogram   [cilindros]   (cilindros_count)

En relación con el atributo `combustible`,

In [12]:
import hvplot.pandas

fig = vehiculos['combustible'].value_counts(
    normalize=True).hvplot.barh(
    ylabel='Proporción',
    xlabel='',
    grid=True,)
fig.opts(
    toolbar='above',
    title='Combustible',
    active_tools=[],
    height=300, width=600
)

:Bars   [combustible]   (proportion)

Se identifica que el atributo `combustible` podría alertar un posible problema en el conjunto de datos al mostrar un 65% de los casos con combustible `Regular`.

### Tratamiento de valores atípicos (_outliers_)

Para detectar valores extremos, una medida general es considerar valores atípicos a todos aquellos valores con una puntuación $z$ (_z-score_) mayor que 3, i.e., todos los valores que se alejan 3 veces o más desviaciones estándar de la media.

El ___z-score___ de una medición $x$ se define como:

\begin{equation*}
z(x)= \frac{x-\mu}{\sigma}
\end{equation*}

donde, $\mu$ y $\sigma$ es la media y la desviación estándar, respectivamente. El _z-score_ índica la cantidad de $\sigma$ que una medición se encuentra alejada de la media.

__IMPORTANTE__: La función [`scipy.stats.zscore`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.zscore.html) no opera adecuadamente con registros nulos. Antes de usarla, se recomienda revisar la documentación para conocer las condiciones que debe cumplir el conjunto de datos, por ejemplo, valores nulos y `NaN`.

In [13]:
def media(series):
    return series.sum() / len(series)

def desviacion_estandar(series):
    cuadrados = (series - media(series)) ** 2
    return ( cuadrados.sum() / (len(series)) ) ** 0.5

def z_score(series):
    return (series - media(series)) / desviacion_estandar(series)

def outliers_by_col(dframe, desviaciones=1):
    for col in dframe:
        # solo columnas numéricas
        if dframe[col].dtype != object:
            outliers = len(dframe[ abs(z_score(dframe[col])) > desviaciones ])
            print(f"{outliers: 4}: {dframe[col].name} ({dframe[col].dtype})")

In [14]:
outliers_by_col(vehiculos, 3)

   0: year (int64)
  47: desplazamiento (float64)
 583: cilindros (float64)
 233: rendimiento (int64)
 358: co2 (float64)


Un _blox plot_ permite detectar de forma gráfica _outliers_. Para este caso, las variables `cilindros`, `desplazamientos`, `rendimiento` y `emision-co2` tienen _outliers_.

In [15]:
import hvplot.pandas

w, h = 180, 300
boxs = vehiculos['co2'].hvplot.box(
    width=w, height=h, grid=True).opts(active_tools=[])
for var in ['rendimiento','desplazamiento','cilindros']:
    box = vehiculos[var].hvplot.box(
        width=w, height=h, grid=True)
    boxs = boxs + box.opts(active_tools=[])
boxs.opts(title='Outliers')

:Layout
   .BoxWhisker.I   :BoxWhisker   (co2)
   .BoxWhisker.II  :BoxWhisker   (rendimiento)
   .BoxWhisker.III :BoxWhisker   (desplazamiento)
   .BoxWhisker.IV  :BoxWhisker   (cilindros)

__RECORDAR__ que, en un un _boxplot_,
- La línea central de la caja representa la mediana.
- La línea inferior de la caja corresponde al primer cuartil.
- La línea superior de la caja corresponde al tercer cuartil.
- Las líneas inferior y superior corresponden a mínimos y máximos (`1.5 IQR`), respectivamente.
- Los puntos son los _outliers_.

Para ejemplificar una verificación de los valores atípicos de la variable `co2`, la detección de valores atípicos (_z-score_ y _box plot_), considerando exclusivamente el atributo `co2`, se puede observar que,

- Existen vehículos que contaminan mucho más de lo normal (criterio $z(x) > 3\sigma$).
- Existen vehículos que contaminan mucho menos de lo normal (criterio $z(x) < -3\sigma$).
- Existen vehículos que no contaminan, es decir, `co2 == 0`.

Para explicar el comportamiento de los valores atípicos, a continuación se identifica que, los vehículos que no contaminan (`co2 = 0`) utilizan como combustible la electricidad.

In [16]:
vehiculos[ vehiculos['co2'] == 0 ]['combustible'].value_counts(normalize=True)

combustible
Electricity    1.0
Name: proportion, dtype: float64

Para __verificar__ que los valores detectados como _outliers_ son efectivamente de este tipo, se evalúa la existencia de estos últimos en el subconjunto de vehículos contaminantes (`co2 > 0`), es decir, aquellos vehículos que no son eléctricos.

In [17]:
no_electricos = vehiculos[ vehiculos['co2'] > 0 ]
outliers_by_col(no_electricos, 3)

   0: year (int64)
  43: desplazamiento (float64)
 583: cilindros (float64)
 400: rendimiento (int64)
 221: co2 (float64)


Considerando que aún existen valores atípicos, se realiza un análisis de dos subconjuntos de vehículos,

- muy contaminantes, puntuación $z(x) > 3$
- poco contaminantes, puntuación $z(x) < -3$

In [18]:
no_electricos.loc[:,'z-score'] = z_score(vehiculos['co2'])
muy_contaminates = no_electricos[ no_electricos['z-score'] > 3 ]
muy_contaminates['combustible'].value_counts()

/var/folders/7w/tzflzn7n1w7bzc10n3mhn9gm0000gn/T/ipykernel_15624/3964358382.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_electricos.loc[:,'z-score'] = z_score(vehiculos['co2'])


combustible
Regular                    131
Premium                     68
Gasoline or natural gas      1
Name: count, dtype: int64

In [19]:
poco_contaminates = no_electricos[no_electricos['z-score'] < -3]
poco_contaminates['combustible'].value_counts()

combustible
Premium Gas or Electricity     13
Regular Gas and Electricity     4
Regular Gas or Electricity      2
Name: count, dtype: int64

Los resultados muestran que los vehículos más contaminantes utilizan gasolina, mientras que los menos contaminantes son híbridos. Por lo tanto, pareciera que los valores atípicos detectados, no lo son. Sin embargo, si se decide filtrar de acuerdo al criterio $3\sigma$,

In [20]:
vehiculos_sin_outliers = vehiculos[( abs(z_score(vehiculos['co2'])) < 3 )]
outliers_by_col(vehiculos_sin_outliers, 3)

   0: year (int64)
  30: desplazamiento (float64)
 535: cilindros (float64)
 386: rendimiento (int64)
  13: co2 (float64)


Removidos los _outliers_, cambia la distribución del conjunto de datos. Luego, se debe verificar la existencia de _outliers_ y se vuelven a eliminar,

In [21]:
vehiculos_sin_outliers = vehiculos_sin_outliers[( abs(z_score(vehiculos_sin_outliers['co2'])) < 3 )]
outliers_by_col(vehiculos_sin_outliers, 3)

   0: year (int64)
  30: desplazamiento (float64)
 535: cilindros (float64)
 375: rendimiento (int64)
   0: co2 (float64)


### Tratamiento de valores inexistentes

Los valores inexistentes podrían ser válidos o no. Es importante, separar estos dos tipos. Por ejemplo, un valor inexistente inválido sería la falta de un apellido de una persona de nacionalidad chilena, sin embargo, para un estadounidense, la inexistencia de su segundo apellido sería válido.

Por otra parte, muchos algoritmos de _Machine Learning_ requieren que no existan valores inexistentes (o `NaN`) en el conjunto de datos objetivo.

Algunas estrategias para tratar dichos valores consisten en,

- Eliminar registros con valores nulos
- Reemplazar o imputar los valores nulos
- Usar un modelo predictivo para predecir los valores inexistentes, a partir de registros anteriores
- [Multivariate imputation by chained equations (MICE)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3074241/) ([implementaciones](https://github.com/iskandr/fancyimpute))
- [Otras estrategias](https://gallery.cortanaintelligence.com/Experiment/Methods-for-handling-missing-values-1)


In [22]:
def nulls_by_col(dframe):
    for atributo in dframe:
        print("{:5}: {} ({})".format(
            len( dframe[ dframe[atributo].isnull() ] ),
            dframe[atributo].name,
            dframe[atributo].dtype
            ))

nulls_by_col(vehiculos)

    0: fabricante (object)
    0: modelo (object)
    0: year (int64)
  140: desplazamiento (float64)
  142: cilindros (float64)
   11: transmision (object)
  797: traccion (object)
    0: clase (object)
    0: combustible (object)
    0: rendimiento (int64)
    0: co2 (float64)


Opción 1. __Eliminar registros__ con valores inexistentes.

[`DataFrame.dropna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)

In [23]:
vehiculos_notnull = vehiculos.dropna(subset=['transmision', 'desplazamiento', 'cilindros', 'traccion'])
nulls_by_col(vehiculos_notnull)

    0: fabricante (object)
    0: modelo (object)
    0: year (int64)
    0: desplazamiento (float64)
    0: cilindros (float64)
    0: transmision (object)
    0: traccion (object)
    0: clase (object)
    0: combustible (object)
    0: rendimiento (int64)
    0: co2 (float64)


Opción 2. __Reemplazar los valores inexistentes con un valor nulo__, por ejemplo, `0` o `nulo`. Es una forma simple de tratar estos valores, pero añade _ruido_ a los datos (se asume que los valores son inexistentes).

[`pandas.DataFrame.fillna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html)

In [24]:
vehiculos['transmision_imp'] = vehiculos['transmision'].fillna('sin transmision')
vehiculos['desplazamiento_imp'] = vehiculos['desplazamiento'].fillna(0)
vehiculos['cilindros_imp'] = vehiculos['cilindros'].fillna(0)

In [25]:
vehiculos[ vehiculos['transmision_imp'] == 'sin transmision']

,fabricante,modelo,year,desplazamiento,cilindros,transmision,traccion,clase,combustible,rendimiento,co2,transmision_imp,desplazamiento_imp,cilindros_imp
13374,Ford,Explorer USPS Electric,2001,NaN,NaN,NaN,2-Wheel Drive,Sport Utility Vehicle - 2WD,Electricity,39,0.000000,sin transmision,0.0,0.0
13375,Ford,Explorer USPS Electric,2002,NaN,NaN,NaN,2-Wheel Drive,Sport Utility Vehicle - 2WD,Electricity,39,0.000000,sin transmision,0.0,0.0
13423,Ford,F150 Pickup 2WD,1984,5.8,8.0,NaN,2-Wheel Drive,Standard Pickup Trucks 2WD,Regular,11,807.909091,sin transmision,5.8,8.0
15209,Ford,Th!nk,2001,NaN,NaN,NaN,NaN,Two Seaters,Electricity,65,0.000000,sin transmision,0.0,0.0
15458,GMC,C25 Pickup 2WD,1984,4.1,6.0,NaN,2-Wheel Drive,Standard Pickup Trucks 2WD,Regular,14,634.785714,sin transmision,4.1,6.0
27984,Nissan,Altra EV,2000,NaN,NaN,NaN,NaN,Midsize Station Wagons,Electricity,85,0.000000,sin transmision,0.0,0.0
28202,Nissan,Hyper-Mini,2001,NaN,NaN,NaN,NaN,Two Seaters,Electricity,75,0.000000,sin transmision,0.0,0.0
35703,Toyota,RAV4 EV,2000,NaN,NaN,NaN,2-Wheel Drive,Sport Utility Vehicle - 2WD,Electricity,72,0.000000,sin transmision,0.0,0.0
35704,Toyota,RAV4 EV,2001,NaN,NaN,NaN,2-Wheel Drive,Sport Utility Vehicle - 2WD,Electricity,72,0.000000,sin transmision,0.0,0.0
35705,Toyota,RAV4 EV,2002,NaN,NaN,NaN,2-Wheel Drive,Sport Utility Vehicle - 2WD,Electricity,78,0.000000,sin transmision,0.0,0.0


In [26]:
# Elimina columnas auxiliares
vehiculos.drop(["transmision_imp", "cilindros_imp", "desplazamiento_imp"], axis=1, inplace=True)

Opción 3. __Reemplazar los valores inexistentes por un valor elegido con un criterio específico__. Prácticas comunes de imputación de datos consiste en reemplazar los valores inexistentes de una variable con su media, moda o mediana.

In [27]:
# registro original
nulls_by_col(vehiculos)

    0: fabricante (object)
    0: modelo (object)
    0: year (int64)
  140: desplazamiento (float64)
  142: cilindros (float64)
   11: transmision (object)
  797: traccion (object)
    0: clase (object)
    0: combustible (object)
    0: rendimiento (int64)
    0: co2 (float64)


In [28]:
# variables categóricas podrían reemplazarse con el valor más frecuente (moda)
transmision_moda = vehiculos['transmision'].mode().values[0] 
traccion_moda = vehiculos['traccion'].mode().values[0]

# variables ordinales, idem
cilindros_moda = vehiculos['cilindros'].mode().values[0]

# variables numéricas podría reemplazarse con la mediana
desplazamiento_mediana = vehiculos['desplazamiento'].median()

# completar datos
vehiculos['traccion'] = vehiculos['traccion'].fillna(traccion_moda)
vehiculos['transmision'] = vehiculos['transmision'].fillna(transmision_moda)
vehiculos['desplazamiento'] = vehiculos['desplazamiento'].fillna(desplazamiento_mediana)
vehiculos['cilindros'] = vehiculos['cilindros'].fillna(cilindros_moda)

nulls_by_col(vehiculos)

    0: fabricante (object)
    0: modelo (object)
    0: year (int64)
    0: desplazamiento (float64)
    0: cilindros (float64)
    0: transmision (object)
    0: traccion (object)
    0: clase (object)
    0: combustible (object)
    0: rendimiento (int64)
    0: co2 (float64)


In [29]:
# Exportar resultados
vehiculos.to_csv("./out/vehiculos-2-limpio_analisis.csv", index=False)

## Actividad

Investigue el uso de una de las siguientes librerías,
- [Vega-Altair](https://altair-viz.github.io/)
- [Seaborn](https://seaborn.pydata.org/)
- [GGplot2](https://ggplot2.tidyverse.org/)
- [Matplotlib](https://matplotlib.org/)
- [Plotly Express](https://plotly.com/python/plotly-express/#overview)

A partir de la fuente de datos utilizada en el presente documento, y las herramientas disponibles en la librería investigada, aborde las siguientes cuestiones:

1. Completar el diagnóstico de calidad de los datos.
2. Mostrar el rendimiento de los vehículos agrupados por categoría (clase).
3. Conclusiones acerca de las emisiones de CO2, por fabricante y categorías de vehículos.


